# Лабораторна робота №5
## Робота з бібліотекою OpenCV

### Завдання:

1. Визначити номер варіанта за формулою:

   N = ord(X) % 5 + 1,
   де X – перша велика літера вашого імені в латинській транскрипції.  
   Наприклад, для James Bond номер варіанта буде N = 5.

2. Завантажити дані з файлу `kr1.xlsx` до датафрейму.

3. Отримати дані до свого варіанту програмно з датафрейму, використовуючи інструменти індексування.

4. Створити функцію, що приймає в якості аргументів дані з датафрейму та повертає зменшене зображення з доданими «круглими окулярами» та зберігає отримане зображення у файл.

### Примітки:
1. При зменшенні зображення помістити обличчя в центрі картинки.
2. Для визначення обличчя та очей використати Haar Cascade з бібліотеки OpenCV.
3. Для малювання використати бібліотеки PIL та/або OpenCV (за бажанням).
4. В якості результату завантажити Jupyter notebook до GitHub.


In [1]:
import pandas as pd
import cv2
from PIL import Image, ImageDraw, ImageColor
import os

file_path = '/home/vlad_322/Documents/Univer/Course3/Python/Lb5/lab6.xlsx'
base_dir = "/home/vlad_322/Documents/Univer/Course3/Python/Lb5/Images"
data = pd.read_excel(file_path)

print(data)

my_variant = ord('V') % 5 + 1
variant_1 = 1
variant_3 = 3
variant_4 = 4
variant_5 = 5
row_for_my_varint = data[data['N'] == my_variant]
row_for_1_varint = data[data['N'] == variant_1]
row_for_3_varint = data[data['N'] == variant_3]
row_for_4_varint = data[data['N'] == variant_4]
row_for_5_varint = data[data['N'] == variant_5]

print(f"\nДані для варіанту {my_variant}:\n{row_for_my_varint}")

def process_image_with_glasses(row, output_dir="output"):
    file_name = row["file name"].iloc[0]
    image_size = tuple(map(int, row["image size"].iloc[0].split('x')))
    glasses_color = ImageColor.getrgb(row["glasses color"].iloc[0])
    line_width = row["line width"].iloc[0]

    image_path = os.path.join(base_dir, file_name)
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Файл {image_path} не знайдено.")

    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    img_height, img_width = image.shape[:2]

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_image)

    face_center = None
    for (x, y, w, h) in faces:
        face_center = (x + w // 2, y + h // 2)
        roi_gray = gray[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(roi_gray)

        eye_centers = []

        for (ex, ey, ew, eh) in eyes:
            if ey < h / 3:
                eye_center = (int(x + ex + ew / 2), int(y + ey + eh / 2))
                radius = int(ew / 2)
                eye_centers.append(eye_center)

                draw.ellipse(
                    [
                        (eye_center[0] - radius, eye_center[1] - radius),
                        (eye_center[0] + radius, eye_center[1] + radius)
                    ],
                    outline=glasses_color,
                    width=line_width
                )

        if len(eye_centers) == 2:
            eye_centers.sort(key=lambda x: x[0])  
            left_eye = eye_centers[0]
            right_eye = eye_centers[1]

            left_inner_edge = (left_eye[0] + radius, left_eye[1])
            right_inner_edge = (right_eye[0] - radius, right_eye[1])

            min_distance = 2 * radius
            distance = abs(left_eye[0] - right_eye[0])

            if distance > min_distance:
                draw.line(
                    [left_inner_edge, right_inner_edge],
                    fill=glasses_color,
                    width=line_width
                )
            left_outer_edge = (left_eye[0] - radius, left_eye[1])
            right_outer_edge = (right_eye[0] + radius, right_eye[1])
            left_ear_point = (left_outer_edge[0] - radius, left_outer_edge[1] + 10)
            right_ear_point = (right_outer_edge[0] + radius, right_outer_edge[1] + 10)

            draw.line([left_outer_edge, left_ear_point], fill=glasses_color, width=line_width)
            draw.line([right_outer_edge, right_ear_point], fill=glasses_color, width=line_width)

    final_image = Image.new("RGB", image_size, (255, 255, 255))

    if face_center:
        face_x, face_y = face_center
        crop_box = (
            max(0, face_x - image_size[0] // 2),
            max(0, face_y - image_size[1] // 2),
            min(img_width, face_x + image_size[0] // 2),
            min(img_height, face_y + image_size[1] // 2)
        )

        cropped_image = pil_image.crop(crop_box)

        final_image.paste(cropped_image, (0, 0))

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_path = os.path.join(output_dir, file_name)
    final_image.save(output_path)

    return output_path

output_path_for_my_variant = process_image_with_glasses(row_for_my_varint)
output_path_for_1_variant = process_image_with_glasses(row_for_1_varint)
output_path_for_3_variant = process_image_with_glasses(row_for_3_varint)
output_path_for_4_variant = process_image_with_glasses(row_for_4_varint)
output_path_for_5_variant = process_image_with_glasses(row_for_5_varint)

print(f"\nЗбережено у файл: {output_path_for_my_variant}")
print(f"\nЗбережено у файл: {output_path_for_1_variant}")
print(f"\nЗбережено у файл: {output_path_for_3_variant}")
print(f"\nЗбережено у файл: {output_path_for_4_variant}")
print(f"\nЗбережено у файл: {output_path_for_5_variant}")

   N         file name image size glasses color  line width
0  1  emma-watson2.jpg    300x300           red           2
1  2   emma-watson.jpg    400x400          blue           3
2  3         draco.jpg    700x700          cyan           4
3  4    longbottom.jpg    600x600       magenta           5
4  5    ron_wesley.jpg    500x500        yellow           6

Дані для варіанту 2:
   N        file name image size glasses color  line width
1  2  emma-watson.jpg    400x400          blue           3

Збережено у файл: output/emma-watson.jpg

Збережено у файл: output/emma-watson2.jpg

Збережено у файл: output/draco.jpg

Збережено у файл: output/longbottom.jpg

Збережено у файл: output/ron_wesley.jpg
